# SteamDB scraping

In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
from selenium import webdriver
import json

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - There is no [win32] chromedriver for browser 102.0.5005 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\gianl\.wdm\drivers\chromedriver\win32\102.0.5005.61]
C:\Users\gianl\AppData\Local\Temp/ipykernel_21008/859633784.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
driver.get('https://steamdb.info/graph/')

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [6]:
games = soup.find_all("tr", class_='app')

In [7]:
id_games=[]
for i in range(len(games)):
    id_games.append(int(games[i]['data-appid']))    #.text -> testo
len(id_games)

## One game

In [10]:
driver.get(f'https://steamdb.info/api/RenderAppHover/?appid={id_games[0]}')

In [11]:
html = driver.page_source
window = BeautifulSoup(html, "lxml")

In [12]:
window

<html><head></head><body><div>
<div class="hover_buttons">
<a class="hover_store" href="https://store.steampowered.com/app/552990/?curator_clanid=4777282&amp;utm_source=SteamDB">Store</a>
<a class="hover_owned" hidden="" href="steam://nav/games/details/552990">In Library</a>
<a class="hover_wishlist" hidden="" href="/extension/">Wishlist</a>
<a class="hover_follow" hidden="" href="/extension/">Follow</a>
<a class="hover_ignore" hidden="" href="/extension/">Ignore</a>
</div>
<a class="hover_title" href="/app/552990/?utm_campaign=SteamDB%20Hover">World of Warships</a>
<div class="js-open-screenshot-viewer" data-screenshots="ss_a1ae80a216ce3dc753f8bcc6750392996ecf3402.jpg,ss_55f768165b614311d0de441d4ed015e2b9496c19.jpg,ss_13192e349cc427dc2fb260528c0626c9b778cf8b.jpg,ss_4447b69df35dd3c374decb659b4aec0097f63d55.jpg,ss_c2d897ec6bbe9391d1492a6e661e12eeefd29236.jpg,ss_8b03fe35e0273a559155159d6b50ba5a1d121de6.jpg,ss_3726bdb5402868368543b43cc3f83152ac2efa47.jpg,ss_7b89d1d56c979e40d36f2a066454794

In [13]:
title = window.find('a',class_='hover_title').text
title

'World of Warships'

In [15]:
informations = {}

for elem in window.find_all('div', class_='hover_body hover_meta'):
    if elem.find('b') is not None:
        informations[elem.text.split(':')[0].strip('\n')] = elem.find('b').text
informations

{'Developer': 'Wargaming Group Limited',
 'Release Date': '15 November 2017',
 'Last Update': '14 June 2022',
 '24h Player Peak': '14,426',
 'Followers': '362,519'}

In [16]:
#<div class="hover_body hover_tag_row">
window.find('div',class_='hover_body hover_tag_row').text.strip().split('\n')

['Naval Combat',
 'Free to Play',
 'Naval',
 'World War II',
 'Massively Multiplayer',
 'Action']

In [20]:
dic = {}
#id game
id_ = id_games[-1]
dic[id_] = {}
driver.get(f'https://steamdb.info/api/RenderAppHover/?appid={id_}')
html = driver.page_source
window = BeautifulSoup(html, "lxml")

#title of game
dic[id_]['title'] = window.find('a',class_='hover_title').text

#info game
for elem in window.find_all('div', class_='hover_body hover_meta'):
    if elem.find('b') is not None:
        dic[id_][elem.text.split(':')[0].strip('\n')] = elem.find('b').text

#categories
categories = window.find('div',class_='hover_body hover_tag_row').text.strip().split('\n')
dic[id_]['categories'] = categories

In [21]:
dic

{686230: {'title': 'OutSplit',
  'Developer': 'Riviysky',
  'Release Date': 'Unknown',
  '24h Player Peak': '0',
  'Followers': '1,139',
  'categories': ['Indie', 'Puzzle']}}

## All games

In [25]:
steam_games = {}
contatore = 0

for i in id_games:
    contatore+=1
    print(contatore, end= "\r")
    
    
    driver.get(f'https://steamdb.info/api/RenderAppHover/?appid={i}')
    html = driver.page_source
    window = BeautifulSoup(html, "lxml")
    
    if 'HTTP ERROR 406' in html:
        pass
    else:
        steam_games[i] = {}
        #title of game
        steam_games[i]['title'] = window.find('a',class_='hover_title').text

        #info game
        for elem in window.find_all('div', class_='hover_body hover_meta'):
            if elem.find('b') is not None:
                steam_games[i][elem.text.split(':')[0].strip('\n')] = elem.find('b').text

        #categories
        if window.find('div',class_='hover_body hover_tag_row') is not None:
            categories = window.find('div',class_='hover_body hover_tag_row').text.strip().split('\n')
            steam_games[i]['categories'] = categories

In [29]:
len(steam_games)

3498

In [30]:
steam_games

{1599340: {'title': 'Lost Ark',
  'Developer': 'Smilegate RPG',
  'Publisher': 'Amazon Games',
  'Release Date': '11 February 2022',
  'Last Update': '9 June 2022',
  '24h Player Peak': '880,727',
  'Followers': '268,276',
  'categories': ['MMORPG',
   'Free to Play',
   'Action RPG',
   'RPG',
   'Action',
   'Hack and Slash']},
 730: {'title': 'Counter-Strike: Global Offensive',
  'Developer': 'Valve',
  'Release Date': '21 August 2012',
  'Last Update': '2 June 2022',
  '24h Player Peak': '840,048',
  'Followers': '3,155,868',
  'categories': ['FPS',
   'Shooter',
   'Multiplayer',
   'Competitive',
   'Action',
   'Team-Based']},
 570: {'title': 'Dota 2',
  'Developer': 'Valve',
  'Release Date': '9 July 2013',
  'Last Update': '13 June 2022',
  '24h Player Peak': '672,736',
  'Followers': '1,981,324',
  'categories': ['Free to Play',
   'MOBA',
   'Multiplayer',
   'Strategy',
   'eSports',
   'Team-Based']},
 578080: {'title': 'PUBG: BATTLEGROUNDS',
  'Developer': 'KRAFTON, Inc.'

### Export

In [41]:
with open('steam_games.json', 'w') as writefile:
    writefile.write(json.dumps(steam_games))